In [2]:
import firebase_admin
from firebase_admin import credentials, storage
from firebase_admin import db
import pandas as pd
import os
import dotenv
dotenv.load_dotenv()

True

### FIREBASE INIT

In [3]:
service_account_info = {
    "type": os.getenv("FIREBASE_TYPE"),
    "project_id": os.getenv("FIREBASE_PROJECT_ID"),
    "private_key_id": os.getenv("FIREBASE_PRIVATE_KEY_ID"),
    "private_key": os.getenv("FIREBASE_PRIVATE_KEY"), 
    "client_email": os.getenv("FIREBASE_CLIENT_EMAIL"),
    "client_id": os.getenv("FIREBASE_CLIENT_ID"),
    "auth_uri": os.getenv("FIREBASE_AUTH_URI"),
    "token_uri": os.getenv("FIREBASE_TOKEN_URI"),
    "auth_provider_x509_cert_url": os.getenv("FIREBASE_AUTH_PROVIDER_X509_CERT_URL"),
    "client_x509_cert_url": os.getenv("FIREBASE_CLIENT_X509_CERT_URL"),
    "universe_domain": os.getenv("FIREBASE_UNIVERSE_DOMAIN")
}

In [4]:
cred = credentials.Certificate(service_account_info) #Create a Firebase credential object using the service account details
firebase_admin.initialize_app(cred, {
    'storageBucket': 'coffee-shop-app-7519c.firebasestorage.app',
    'databaseURL': 'https://coffee-shop-app-7519c-default-rtdb.firebaseio.com/'
})

In [5]:
bucket = storage.bucket() #Get a reference to the Firebase Storage bucket

### Upload Data

In [13]:
image_folder_path = './products/images/' #Path to the folder containing images

In [14]:
products_collection = db.reference('products') #Reference to the 'products' collection in Firebase Realtime Database

In [15]:
df = pd.read_json('products/products.jsonl', lines=True) #Read the JSONL file into a DataFrame
df.head(2) #Display the first few rows of the DataFrame

,name,category,description,ingredients,price,rating,image_path
0,Cappuccino,Coffee,A rich and creamy cappuccino made with freshly...,"[Espresso, Steamed Milk, Milk Foam]",4.50,4.7,cappuccino.jpg
1,Jumbo Savory Scone,Bakery,"Deliciously flaky and buttery, this jumbo savo...","[Flour, Butter, Cheese, Herbs, Baking Powder, ...",3.25,4.3,SavoryScone.webp


In [16]:
def upload_image(bucket, image_path):
    image_name = image_path.split('/')[-1] #Extract the image name from the path
    blob = bucket.blob(f'product_images/{image_name}') #Create a blob object for the image in Firebase Storage
    #Upload the image
    blob.upload_from_filename(image_path)
    #Make the image publicly accessible and get its URL
    blob.make_public()
    return blob.public_url #Return the public URL of the uploaded image

In [ ]:
# Loops through each product, uploads image, adds URL, and pushes to Firebase DB.

for index, row in df.iterrows():
    print(index, row['name'])

    image_path = os.path.join(image_folder_path, row['image_path']) #Construct the full path to the image

    image_url = upload_image(bucket, image_path) #Upload the image and get its URL
    product_data = row.to_dict() #Convert the row to a dictionary
    product_data.pop('image_path') #Remove the 'image_path' key from the dictionary
    product_data['image_url'] = image_url #Add the image URL to the dictionary
    #Add to firebase
    products_collection.push().set(product_data) #Push the product data to Firebase Realtime Database

0 Cappuccino
1 Jumbo Savory Scone
2 Latte
3 Chocolate Chip Biscotti
4 Espresso shot
5 Hazelnut Biscotti
6 Chocolate Croissant
7 Dark chocolate
8 Cranberry Scone
9 Croissant
10 Almond Croissant
11 Ginger Biscotti
12 Oatmeal Scone
13 Ginger Scone
14 Chocolate syrup
15 Hazelnut syrup
16 Carmel syrup
17 Sugar Free Vanilla syrup
